## Main file to calibrate EBSM models for Tairua data

In [ ]:
# Loading libraries

import os
import scipy.io
import numpy as np
import pandas as pd
import scipy.optimize as opt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.optimize import minimize
import scipy.interpolate as interpolate

os.chdir('..')
os.chdir('./modules')
from calibration import *
from equilibriumModels import *
os.chdir('..')
os.chdir('./src')

plt.rcParams.update({'font.family': 'serif'})
plt.rcParams.update({'font.size': 7})
plt.rcParams.update({'font.weight': 'bold'})
font = {'family': 'serif',
        'weight': 'bold',
        'size': 8}

In [ ]:
# Load data
os.chdir('..')
os.chdir('./data')
waves = scipy.io.loadmat("Wave_hindcast_corrected.mat")
ss_past = scipy.io.loadmat("hindcast_SS_corr.mat")
ss_fut = scipy.io.loadmat("forecast_SS.mat")
shorelines = scipy.io.loadmat("Shorecast.mat")
at = scipy.io.loadmat("Tide_past.mat")
os.chdir('..')
os.chdir('./src')

# Organize variables

timesf = ss_fut["time"]
Storm_Surge = ss_fut["Storm_Surge"]
Storm_surge = ss_past["Storm_surge"]
hindcast = waves["hindcast"]
Tide_past = at["Tide_past"]
Shorecast = shorelines["Shorecast"]

In [ ]:
# Setting calibration period
dt = 3.0
times = np.arange(Shorecast["time"][0][0][0], Shorecast["time"][0][0][-1] + dt / 24, dt / 24)

AT = interpolate.interp1d(np.squeeze(Tide_past["time"][0][0]), np.squeeze(Tide_past["tide"][0][0]))(times)
SS = interpolate.interp1d(np.concatenate([np.squeeze(Storm_surge["time"][0][0]), np.squeeze(timesf)]), 
                          np.concatenate([np.squeeze(Storm_surge["SS"][0][0]), np.squeeze(Storm_Surge)]))(times)
Hs = interpolate.interp1d(np.squeeze(hindcast["time"][0][0]), np.squeeze(hindcast["Hs"][0][0]))(times)
Tp = interpolate.interp1d(np.squeeze(hindcast["time"][0][0]), np.squeeze(hindcast["Tp"][0][0]))(times)
theta = interpolate.interp1d(np.squeeze(hindcast["time"][0][0]), np.squeeze(hindcast["Dir"][0][0]))(times)

ENS = {"Yobs": np.squeeze(Shorecast["average"][0][0]), "time": np.squeeze(Shorecast["time"][0][0])}

ENS["dates"] = pd.to_datetime(ENS["time"]-719529,unit='d').round('s').to_pydatetime()

dates = pd.to_datetime(times-719529,unit='d').round('s').to_pydatetime()

In [ ]:
# Params setup
d50 = 0.3e-3
Hberm = 1
Yi = ENS["Yobs"][0]
flagP = 4
depth = 10
angleBathy = 54.3

indexer = np.vectorize(lambda i: np.argmin(np.abs(times - ENS["time"][i])))

ENS["indexes"] = indexer(np.arange(0,len(ENS["time"])))

In [ ]:
# Initialize class for running the models
Setup = EBSM(times, Hs, SS, AT, Tp, d50)
Setup.LinearBreak(theta, depth, angleBathy)
Setup.MillerDean(dt, Yi, Hberm, flagP)
Setup.Yates09(dt, Yi)
Setup.ShoreFor(dt, Yi)

ngen = 5000
npop = 100
mag = 0.5

In [ ]:
print("Starting Miller and Dean 2004...")

npar = 3

def obj_md(X):
    return Objective("MD", Setup, X, "RMSE", ENS)
x0_md = np.log([78.75, 6.9211e-05, 8.4790e-04])
lb_md = np.log([60, 5e-6, 5e-5])
ub_md = np.log([100, 1e-3, 8e-3])
params_md, met_md = sce_ua2(obj_md, x0_md, ngen, npop, npar, mag, lb_md, ub_md)

Y_md = millerDean04(Setup, np.exp(params_md))

In [7]:
YY, yeq= millerDean04_jit(Setup.Hb, Setup.MD["sl"], Setup.depthb, Setup.Omega, 
                             Setup.MD["flagP"], Setup.Wast, Setup.MD["Hberm"], Setup.MD["dt"],
                             Setup.MD["Yi"], [78.75, 6.9211e-05, 8.4790e-04])

YYsl = YY[ENS["indexes"]]

np.sqrt(np.mean((YYsl - ENS["Yobs"])**2))

c:\Users\freitasl\.conda\envs\PyJu\lib\site-packages\numba\core\ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'params' of function 'millerDean04_jit'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\modules\equilibriumModels.py", line 191:
@jit(nopython = True)
def millerDean04_jit(Hb, sl, depthb, Omega, flagP, Wast, Hberm, dt, Yi, params):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


ValueError: too many values to unpack (expected 2)

In [ ]:
print("Starting Yates et al. 2009...")

npar = 4

def obj_y09(X):
    return Objective("Y09", Setup, X, "RMSE", ENS)
x0_y09 = np.log([0.1143, 9.6392, 0.0034, 0.0038])
lb_y09 = np.log([0.01, 1, 1e-4, 0.001])
ub_y09 = np.log([0.3, 15, 0.05, 0.01])
params_y09, met_y09 = sce_ua2(obj_y09, x0_y09, ngen, npop, npar, mag, lb_y09, ub_y09)
Y_y09 = yates09(Setup, np.exp(params_y09))

In [ ]:
print("Starting ShoreFor...")

npar = 3

def obj_sf(X):
    return Objective("SF", Setup, X, "RMSE", ENS)
x0_sf = np.log([100, 3.7057e-05, 64])
lb_sf = np.log([60, 0.5e-5, 50])
ub_sf = np.log([200, 5e-3, 80])
params_sf, met_sf = sce_ua2(obj_sf, x0_sf, ngen, npop, npar, mag, lb_sf, ub_sf)
Y_sf = shorefor(Setup, np.exp(params_sf))

In [ ]:
plt.figure(figsize=(10, 2), dpi=300, linewidth=5, edgecolor="#04253a")
plt.scatter(ENS["dates"], ENS["Yobs"], s = 1, c = 'grey', label = 'Observed data')
plt.plot(dates, Y_md, lw = 0.5, color = 'b', label = r'Miller and Dean 2004')
plt.plot(dates, Y_y09, lw = 0.5, color = 'r', label = r'Yates et al. 2009')
plt.plot(dates, Y_sf, lw = 0.5, color = 'k', label = r'Davidson et al. 2013')
plt.ylabel('Y [m]', fontdict=font)
plt.legend(ncol = 4,prop={'size': 6}, loc = 'upper center')
plt.xlim((dates[0], dates[-1]))
plt.grid(visible=True, which='both', linestyle = '--', linewidth = 0.5)
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter(''))